In [5]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import lxml
import requests
import re
import pandas as pd
import numpy as np
from lxml import html
import csv

In [6]:
# Needs to be changed to 201 to 301
for i in range(1,2):
    
    base_url = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page="+str(i)

    html = urlopen(base_url)
    soup = BeautifulSoup(html.read(), features="lxml")

    links = []
    regex = "/book/show/."

    for link in soup.find_all("a"):
        if re.match(regex, str(link.get("href"))):
            links.append("https://www.goodreads.com"+link.get("href"))
    
    #issues of duplicates        
    links = list(dict.fromkeys(links))

    outfile = open("test.txt", "w")
    for link in links:
        outfile.write(link)
        outfile.write("\n")
    outfile.close()

In [46]:
# Functions
def get_title(soup):
    # This one below does not work for all the urls...I have no idea why
    #title = soup.find('h1', {'id': 'bookTitle'})
    # This one works for ALL urls but the format is not great
    title = soup.find("title").get_text()
    return (title)


def get_series(soup):
    if soup.find('h2', {'id': "bookSeries"}):
        series = soup.find('h2', {'id': "bookSeries"}).text.strip()
        return (series)
    return ''    

def get_author(soup):
    if soup.find('span', {'itemprop': "author"}):
        author = soup.find('span', {'itemprop': "author"}).text.strip()
        return (author)
    return ''   

def get_num_pages(soup):
    if soup.find('span', {'itemprop': 'numberOfPages'}):
        num_pages = soup.find('span', {'itemprop': 'numberOfPages'}).text.strip()
        return int(num_pages.split()[0])
    return ''

def get_rating_value(soup):
    if soup.find('span', {'itemprop': 'ratingValue'}):
        rating_value = soup.find('span', {'itemprop': 'ratingValue'}).text.strip()
        return float(rating_value)
    return ''

def get_rating_count(soup):
    if soup.find('span', {'itemprop': 'ratingCount'}):
        rating_count = soup.find('span', {'itemprop': 'ratingCount'}).text.strip()
        return int(rating_count)
    return ''

def get_review_count(soup):
    if soup.find('meta', {'itemprop': 'reviewCount'}, ):
        review_count = soup.find('meta', {'itemprop': 'reviewCount'})
        # this does not convert to int
        return (str(review_count).split()[1].replace("content=",""))
    return ""

def get_year_first_published(soup):
    year_first_published = soup.find('nobr', attrs={'class':'greyText'}).string
    return re.search('([0-9]{3,4})', year_first_published).group(1)

def get_plot(soup):
    plot = soup.find("div", {"id":"description"}).text.strip()
    return (plot)

# Work for SOME URLs but not others....wtf
#html = urlopen("https://www.goodreads.com/book/show/25439754-before-life-happened")
html = urlopen("https://www.goodreads.com/book/show/2767052-the-hunger-games")
#html = urlopen("https://www.goodreads.com/book/show/2.Harry_Potter_and_the_Order_of_the_Phoenix")

soup = BeautifulSoup(html.read(), features="lxml")

print(soup)

<!DOCTYPE html>
<html class="desktop withSiteHeaderTopFullImage">
<head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# good_reads: http://ogp.me/ns/fb/good_reads#">
<title>The Hunger Games (The Hunger Games, #1) by Suzanne Collins</title>
<meta content="The Hunger Games book. Read 172,593 reviews from the world's largest community for readers. Could you survive on your own in the wild, with every one out..." name="description"/>
<meta content="telephone=no" name="format-detection"/>
<link href="https://www.goodreads.com/book/show/2767052-the-hunger-games" rel="canonical"/>
<meta content="2415071772" property="fb:app_id"/>
<meta content="books.book" property="og:type"/>
<meta content="The Hunger Games (The Hunger Games, #1)" property="og:title"/>
<meta content="Could you survive on your own in the wild, with every one out to make sure you don't live to see the morning?  In the ruins of a place on..." property="og:description"/>
<meta content="https://i.gr-assets.com/images/S/com

In [26]:
# not working
with open('test.tsv', 'w') as f_output:
    with open("test.txt", "r", encoding="utf-8") as outfile:
        for url in outfile[]:
            html = urlopen(url)
            soup = BeautifulSoup(html.read(), features="lxml")
            test_list_tsv = get_title(soup)
            tsv_output = csv.writer(f_output, delimiter='\t')
            tsv_output.writerow(test_list_tsv)

https://www.goodreads.com/book/show/2767052-the-hunger-games

https://www.goodreads.com/book/show/2.Harry_Potter_and_the_Order_of_the_Phoenix

https://www.goodreads.com/book/show/2657.To_Kill_a_Mockingbird

https://www.goodreads.com/book/show/1885.Pride_and_Prejudice

https://www.goodreads.com/book/show/41865.Twilight

https://www.goodreads.com/book/show/19063.The_Book_Thief



KeyboardInterrupt: 

Alternative Way to find the information, not very accurate. 

In [ ]:
outfile = open("test.txt", "r", encoding="utf-8")

url = outfile.readlines()[1]

page = requests.get(url)
tree = html.fromstring(page.content)

#Problem is that this creates list object, and not very clean one
bookTitle = tree.xpath("//h1[@id='bookTitle']/text()")[0].strip()
bookSeries = tree.xpath("//h2[@id='bookSeries']/text()")[0].strip()
bookAuthors = tree.xpath("//span[@itemprop='name']/text()")[0].strip()
NumberofPages = tree.xpath("//span [@itemprop='numberOfPages']/text()")
ratingValue = tree.xpath("//span [@itemprop='ratingValue']/text()")[0].strip()
ratingCount = tree.xpath("//*[@id='bookMeta']/a[2]/text()")[0].strip()
reviewCount = tree.xpath("//*[@id='bookMeta']/a[3]/text()")[0].strip()
PublishingDate = " ".join(tree.xpath("//div [@class='row']/text()")[0].split()[1:])

test_list_tsv = [bookTitle, bookSeries, bookAuthors, ratingValue, ratingCount, reviewCount, PublishingDate]